In [3]:
#cargamos las librerias que vamos a necesitar:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px

In [4]:
# cargamos nuestro dataset:
pathmac = "/Users/fersaol/Library/CloudStorage/OneDrive-Personal/Documentos/MIS_COSAS_DE_THE_BRIDGE/Proyectos_Machine_Learning/co2_ml/src/data/raw/df_40_final_v3.csv"
pathwin = "C:/Users/ferna/OneDrive/Documentos/MIS_COSAS_DE_THE_BRIDGE/EDA/proyecto_co2/src/data/df_40_final_v3.csv"
df = pd.read_csv(pathwin)
df = df.drop(columns="Unnamed: 0")
df.head()

,GDP,Population,Energy_production,Energy_consumption,CO2_emission,per_capita_production,Energy_intensity_by_GDP,balance,eficiency,energy_dependecy,use_intensity_pc,co2_pc,latitude,longitude,Year,Country,Energy_type,CODE_x,continent,geometry
0,279.389,15182.60,0.150,0.155,0.0,104.341,5.670,-0.005,0.0,0.001,0.0,0.0,6.424,-66.59,1980-01-01,Venezuela,renewables_n_other,None,South America,NaN
1,279.389,15182.60,5.000,0.812,0.0,104.341,5.670,4.188,0.0,0.003,0.0,0.0,6.424,-66.59,1980-01-01,Venezuela,petroleum_n_other_liquids,None,South America,NaN
2,279.389,15182.60,0.616,0.616,0.0,104.341,5.670,0.000,0.0,0.002,0.0,0.0,6.424,-66.59,1980-01-01,Venezuela,natural_gas,None,South America,NaN
3,279.389,15182.60,0.001,0.002,0.0,104.341,5.670,-0.000,0.0,0.000,0.0,0.0,6.424,-66.59,1980-01-01,Venezuela,coal,None,South America,NaN
4,278.376,15597.88,0.156,0.161,0.0,110.257,6.178,-0.005,0.0,0.001,0.0,0.0,6.424,-66.59,1981-01-01,Venezuela,renewables_n_other,None,South America,NaN


Vamos a eliminar la columna geometry, puesto que no nos va ha hacer falta para nuestros modelos o representación de los datos:

In [5]:
df = df.drop(columns="geometry")
df.head(2)

,GDP,Population,Energy_production,Energy_consumption,CO2_emission,per_capita_production,Energy_intensity_by_GDP,balance,eficiency,energy_dependecy,use_intensity_pc,co2_pc,latitude,longitude,Year,Country,Energy_type,CODE_x,continent
0,279.389,15182.6,0.15,0.155,0.0,104.341,5.67,-0.005,0.0,0.001,0.0,0.0,6.424,-66.59,1980-01-01,Venezuela,renewables_n_other,None,South America
1,279.389,15182.6,5.00,0.812,0.0,104.341,5.67,4.188,0.0,0.003,0.0,0.0,6.424,-66.59,1980-01-01,Venezuela,petroleum_n_other_liquids,None,South America


Vemos que en la variable CODE_x nos faltan los códigos de algunos países, vamos a ver de cuales:

In [6]:
df.loc[df.CODE_x == "None","Country"].unique()

array(['Venezuela', 'Iran', 'Russia', 'Former Czechoslovakia',
       'Former Yugoslavia'], dtype=object)

In [7]:
df.Country.unique()

array(['Venezuela', 'Iran', 'Russia', 'Former Czechoslovakia',
       'Former Yugoslavia', 'Norway', 'Oman', 'Netherlands', 'Nigeria',
       'Qatar', 'Poland', 'Malaysia', 'Libya', 'Mexico', 'United States',
       'United Kingdom', 'United Arab Emirates', 'Saudi Arabia',
       'South Africa', 'Canada', 'Colombia', 'China', 'Argentina',
       'Australia', 'Algeria', 'Angola', 'Brazil', 'Indonesia', 'India',
       'Kuwait', 'Iraq', 'Japan', 'Egypt', 'Germany', 'France',
       'Turkmenistan', 'Ukraine', 'Uzbekistan', 'Kazakhstan'],
      dtype=object)

In [8]:
len(df.groupby("Country").mean())

39

En el caso de Former Yugoslavia, es una entidad extinguida en el año 2003, que desde el 1991 estaba compuesta por Serbia y por Montenegro. Sin embargo, en los años anteriores al 1991 estaba compuesta por otros tantos países como son la República Socialista de Bosnia y Herzegovina, República Socialista de Croacia, República Socialista de Eslovenia, República Socialista de Macedonia, República Socialista de Montenegro y la República Socialista de Serbia. Como nos es imposible diferenciar que parte de los datos corresponde a cada uno de ls distintos países que compusieron a los distintos agregados nacionales hemos decidido eliminarlo del dataset.

In [9]:
df = df.drop(index=df[df.Country == "Former Yugoslavia"].index)

In [10]:
df.Country.unique()

array(['Venezuela', 'Iran', 'Russia', 'Former Czechoslovakia', 'Norway',
       'Oman', 'Netherlands', 'Nigeria', 'Qatar', 'Poland', 'Malaysia',
       'Libya', 'Mexico', 'United States', 'United Kingdom',
       'United Arab Emirates', 'Saudi Arabia', 'South Africa', 'Canada',
       'Colombia', 'China', 'Argentina', 'Australia', 'Algeria', 'Angola',
       'Brazil', 'Indonesia', 'India', 'Kuwait', 'Iraq', 'Japan', 'Egypt',
       'Germany', 'France', 'Turkmenistan', 'Ukraine', 'Uzbekistan',
       'Kazakhstan'], dtype=object)

Para el caso de Former Czechoslovakia hemos decidio que al ser la escisión de solo dos países, República Checa y Eslovaquia vamos a dividir sus datos de manera proporcional a la extensión en km2 que poseea cada uno de los países. De esta manera a:
- la República Checa le corresponderán el: 
$$ \frac{78871\ km^2}{78871\ km^2 + 49035\ km^2} = 0.6167 $$
- a Eslovaquia le corresponderá el: 
$$ \frac{49035\ km^2}{78871\ km^2 + 49035\ km^2} = 0.3833 $$

In [11]:
fc = df[df.Country == "Former Czechoslovakia"]
fc_num = fc.select_dtypes("number")
fc_obj = fc.select_dtypes("object")


In [12]:
fc_num.head(2)

,GDP,Population,Energy_production,Energy_consumption,CO2_emission,per_capita_production,Energy_intensity_by_GDP,balance,eficiency,energy_dependecy,use_intensity_pc,co2_pc,latitude,longitude
529,116.352,4781.598,0.050,-0.012,0.0,0.0,0.0,0.062,0.0,-0.0,-0.0,0.0,49.817492,15.472962
530,116.352,4781.598,0.055,0.055,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,49.817492,15.472962


In [13]:
len(fc) == len(df[df.Country == "Former Czechoslovakia"])

True

In [14]:
fc_num_es = fc_num * 0.3833
fc_num_es.head(2)

,GDP,Population,Energy_production,Energy_consumption,CO2_emission,per_capita_production,Energy_intensity_by_GDP,balance,eficiency,energy_dependecy,use_intensity_pc,co2_pc,latitude,longitude
529,44.597722,1832.786513,0.019165,-0.004600,0.0,0.0,0.0,0.023765,0.0,-0.0,-0.0,0.0,19.095045,5.930786
530,44.597722,1832.786513,0.021081,0.021081,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,19.095045,5.930786


In [15]:
fc_num_rc = fc_num * 0.6167
fc_num_rc.head(2)

,GDP,Population,Energy_production,Energy_consumption,CO2_emission,per_capita_production,Energy_intensity_by_GDP,balance,eficiency,energy_dependecy,use_intensity_pc,co2_pc,latitude,longitude
529,71.754278,2948.811487,0.030835,-0.007400,0.0,0.0,0.0,0.038235,0.0,-0.0,-0.0,0.0,30.722447,9.542176
530,71.754278,2948.811487,0.033919,0.033919,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,30.722447,9.542176


In [16]:
fc_obj.head(2)

,Year,Country,Energy_type,CODE_x,continent
529,1980-01-01,Former Czechoslovakia,renewables_n_other,None,Europe
530,1980-01-01,Former Czechoslovakia,nuclear,None,Europe


In [17]:
fc_obj_es = fc_obj.copy()
fc_obj_es["Country"] = fc_obj.Country.map({"Former Czechoslovakia":"Slovakia"})
fc_obj_es.head(2)

,Year,Country,Energy_type,CODE_x,continent
529,1980-01-01,Slovakia,renewables_n_other,None,Europe
530,1980-01-01,Slovakia,nuclear,None,Europe


In [18]:
fc_obj_es["CODE_x"] = fc_obj.CODE_x.map({"None":"SK"})
fc_obj_es.head(2)

,Year,Country,Energy_type,CODE_x,continent
529,1980-01-01,Slovakia,renewables_n_other,SK,Europe
530,1980-01-01,Slovakia,nuclear,SK,Europe


In [19]:
fc_obj_es = fc_obj_es.reset_index().drop(columns="index")
fc_num_es = fc_num_es.reset_index().drop(columns="index")
df_sk = pd.merge(fc_num_es,fc_obj_es,left_index=True,right_index=True)
df_sk.head()

,GDP,Population,Energy_production,Energy_consumption,CO2_emission,per_capita_production,Energy_intensity_by_GDP,balance,eficiency,energy_dependecy,use_intensity_pc,co2_pc,latitude,longitude,Year,Country,Energy_type,CODE_x,continent
0,44.597722,1832.786513,0.019165,-0.004600,0.0,0.0,0.0,0.023765,0.0,-0.000000,-0.0,0.0,19.095045,5.930786,1980-01-01,Slovakia,renewables_n_other,SK,Europe
1,44.597722,1832.786513,0.021081,0.021081,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,19.095045,5.930786,1980-01-01,Slovakia,nuclear,SK,Europe
2,44.597722,1832.786513,0.001917,0.298207,0.0,0.0,0.0,-0.296674,0.0,0.002683,0.0,0.0,19.095045,5.930786,1980-01-01,Slovakia,petroleum_n_other_liquids,SK,Europe
3,44.597722,1832.786513,0.007283,0.128406,0.0,0.0,0.0,-0.121506,0.0,0.001150,0.0,0.0,19.095045,5.930786,1980-01-01,Slovakia,natural_gas,SK,Europe
4,44.597722,1832.786513,0.758167,0.752801,0.0,0.0,0.0,0.005366,0.0,0.006516,0.0,0.0,19.095045,5.930786,1980-01-01,Slovakia,coal,SK,Europe


In [20]:
fc_obj_rc = fc_obj.copy()
fc_obj_rc["Country"] = fc_obj.Country.map({"Former Czechoslovakia":"Czech Republic"})
fc_obj_rc.head(2)

,Year,Country,Energy_type,CODE_x,continent
529,1980-01-01,Czech Republic,renewables_n_other,None,Europe
530,1980-01-01,Czech Republic,nuclear,None,Europe


In [21]:
fc_obj_rc["CODE_x"] = fc_obj.CODE_x.map({"None":"CZE"})
fc_obj_rc.head(2)

,Year,Country,Energy_type,CODE_x,continent
529,1980-01-01,Czech Republic,renewables_n_other,CZE,Europe
530,1980-01-01,Czech Republic,nuclear,CZE,Europe


In [22]:
fc_obj_rc = fc_obj_rc.reset_index().drop(columns="index")
fc_num_rc = fc_num_rc.reset_index().drop(columns="index")
df_cz = pd.merge(fc_num_rc,fc_obj_rc,left_index=True,right_index=True)
df_cz.head()

,GDP,Population,Energy_production,Energy_consumption,CO2_emission,per_capita_production,Energy_intensity_by_GDP,balance,eficiency,energy_dependecy,use_intensity_pc,co2_pc,latitude,longitude,Year,Country,Energy_type,CODE_x,continent
0,71.754278,2948.811487,0.030835,-0.007400,0.0,0.0,0.0,0.038235,0.0,-0.000000,-0.0,0.0,30.722447,9.542176,1980-01-01,Czech Republic,renewables_n_other,CZE,Europe
1,71.754278,2948.811487,0.033919,0.033919,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,30.722447,9.542176,1980-01-01,Czech Republic,nuclear,CZE,Europe
2,71.754278,2948.811487,0.003084,0.479793,0.0,0.0,0.0,-0.477326,0.0,0.004317,0.0,0.0,30.722447,9.542176,1980-01-01,Czech Republic,petroleum_n_other_liquids,CZE,Europe
3,71.754278,2948.811487,0.011717,0.206595,0.0,0.0,0.0,-0.195494,0.0,0.001850,0.0,0.0,30.722447,9.542176,1980-01-01,Czech Republic,natural_gas,CZE,Europe
4,71.754278,2948.811487,1.219833,1.211199,0.0,0.0,0.0,0.008634,0.0,0.010484,0.0,0.0,30.722447,9.542176,1980-01-01,Czech Republic,coal,CZE,Europe


In [23]:
df = pd.concat([df,df_sk,df_cz],axis=0)
df.isnull().sum()

GDP                        0
Population                 0
Energy_production          0
Energy_consumption         0
CO2_emission               0
per_capita_production      0
Energy_intensity_by_GDP    0
balance                    0
eficiency                  0
energy_dependecy           0
use_intensity_pc           0
co2_pc                     0
latitude                   0
longitude                  0
Year                       0
Country                    0
Energy_type                0
CODE_x                     0
continent                  0
dtype: int64

In [24]:
df = df.drop(df[df.Country == "Former Czechoslovakia"].index,axis=0)

In [25]:
df = df.reset_index().drop(columns="index")

Comprobamos si tenemos algunos nones adicionales:

In [26]:
lista_paises = df[df.CODE_x == "None"]["Country"].unique()

In [27]:
df.loc[0,"Country"]

'Venezuela'

In [28]:
def rellenado(df,paises,iso):

    for i in range(len(df)):
        if df.loc[i,"Country"] == paises[0]:
            df.loc[i,"CODE_x"] = iso[0]
        elif df.loc[i,"Country"] == paises[1]:
            df.loc[i,"CODE_x"] = iso[1]
        elif df.loc[i,"Country"] == paises[2]:
            df.loc[i,"CODE_x"] = iso[2]
            


In [29]:
rellenado(df,lista_paises,["VEN","IRN","RUS"])

In [30]:
X = df.drop(['GDP',"Year","latitude","longitude","Country","Energy_type","CODE_x","continent"], axis=1)
y = df['GDP'].copy()

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [32]:
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestRegressor
rfecv = RFECV(estimator=RandomForestRegressor(),step=1,cv=10,scoring="neg_mean_absolute_error")
rfecv.fit(X_train,y_train)
print("Número óptimo de variables: %d" % rfecv.n_features_)
print('Variables a seleccionar: %s' % list(X_train.columns[rfecv.support_]))
print(rfecv.ranking_)

Número óptimo de variables: 4
Variables a seleccionar: ['Population', 'Energy_consumption', 'per_capita_production', 'Energy_intensity_by_GDP']
[1 6 1 3 1 1 5 2 4 8 7]


In [37]:
df2 = px.data.gapminder().query("year==2007")
fig = px.choropleth(df, locations="CODE_x",
                    color="CO2_emission", # lifeExp is a column of gapminder
                    hover_name="Country", # column to add to hover information
                    color_continuous_scale=px.colors.sequential.Plasma)
fig.show()

In [34]:
df.CODE_x.unique()

array(['VEN', 'IRN', 'RUS', 'NOR', 'OMN', 'NLD', 'NGA', 'QAT', 'POL',
       'MYS', 'LBY', 'MEX', 'USA', 'GBR', 'ARE', 'SAU', 'ZAF', 'CAN',
       'COL', 'CHN', 'ARG', 'AUS', 'DZA', 'AGO', 'BRA', 'IDN', 'IND',
       'KWT', 'IRQ', 'JPN', 'EGY', 'DEU', 'FRA', 'TKM', 'UKR', 'UZB',
       'KAZ', 'SK', 'CZE'], dtype=object)

Guardamos una copia de seguridad del dataframe procesado:

In [35]:
# path_saving = "/Users/fersaol/Library/CloudStorage/OneDrive-Personal/Documentos/MIS_COSAS_DE_THE_BRIDGE/Proyectos_Machine_Learning/co2_ml/src/data/processed/"
# name = "df_v3.csv"
# df.to_csv(path_saving + name,header=True,index=False)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/fersaol/Library/CloudStorage/OneDrive-Personal/Documentos/MIS_COSAS_DE_THE_BRIDGE/Proyectos_Machine_Learning/co2_ml/src/data/processed/df_v3.csv'